In [ ]:
#Emad Gholibeigi

In [ ]:
# make convolotion layers
from keras.layers import Conv2D , Dense , Dropout , Flatten , MaxPooling2D ,BatchNormalization
# to make sequential model
from keras.models import Sequential
# import cifar10 dataset
from keras.datasets import cifar10
import tensorflow as tf
import numpy as np                                
import matplotlib.pyplot as plt
import keras as k
from keras.optimizers import SGD, Adam
from keras.regularizers import l2
import h5py
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import scipy as scipy

In [ ]:
from random import randrange

In [ ]:
from keras.utils import np_utils as cat

In [ ]:
( x_train , y_train ) , ( x_test , y_test ) = cifar10.load_data()
img_rows, img_cols , channels= 32,32,3

In [ ]:
# set up image augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    horizontal_flip=True,
    width_shift_range=0.15,
    height_shift_range=0.15
    #zoom_range=0.3
    )
datagen.fit(x_train)

In [ ]:
#reshape into images
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, channels)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, channels)
input_shape = (img_rows, img_cols, 1)
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
# labels
num_classes=10
y_train = k.utils.to_categorical(y_train, num_classes)
y_test = k.utils.to_categorical(y_test, num_classes)

In [ ]:
# plotting helper function
def plothist(hist):
    plt.plot(hist.history['accuracy'])
    plt.plot(hist.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
#convert integers to float; normalise and center the mean
x_train=x_train.astype("float32")  
x_test=x_test.astype("float32")
mean=np.mean(x_train)
std=np.std(x_train)
x_test=(x_test-mean)/std
x_train=(x_train-mean)/std

In [ ]:
reg2=None
num_filters2=32
ac2='relu'
adm2=Adam(learning_rate=0.0025, epsilon=1e-08)
opt=adm2
drop_dense2=0.5
drop_conv2=0.2

model = Sequential()

model.add(Conv2D(num_filters2, (3, 3), activation=k.layers.LeakyReLU(alpha=0.01),
                 kernel_regularizer=reg2, input_shape=(img_rows, img_cols, channels),padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(num_filters2, (3, 3), activation=k.layers.LeakyReLU(alpha=0.01),kernel_regularizer=reg2,padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))   # reduces to 16x16x3xnum_filters
model.add(Dropout(drop_conv2))

model.add(BatchNormalization(axis=-1))
model.add(Conv2D(2*num_filters2, (3, 3), activation=k.layers.LeakyReLU(alpha=0.01),kernel_regularizer=reg2,padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(2*num_filters2, (3, 3), activation=k.layers.LeakyReLU(alpha=0.01),kernel_regularizer=reg2,padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))  # reduces to 8x8x3x(2*num_filters)
model.add(Dropout(drop_conv2))

model.add(BatchNormalization(axis=-1))
model.add(Conv2D(4*num_filters2, (3, 3), activation=k.layers.LeakyReLU(alpha=0.01),kernel_regularizer=reg2,padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(4*num_filters2, (3, 3), activation=k.layers.LeakyReLU(alpha=0.01),kernel_regularizer=reg2,padding='same'))
model.add(BatchNormalization(axis=-1))  
model.add(MaxPooling2D(pool_size=(2, 2)))  # reduces to 4x4x3x(4*num_filters)
model.add(Dropout(drop_conv2))

model.add(BatchNormalization(axis=-1))
model.add(Conv2D(8*num_filters2, (3, 3), activation=k.layers.LeakyReLU(alpha=0.01),kernel_regularizer=reg2,padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(8*num_filters2, (3, 3), activation=k.layers.LeakyReLU(alpha=0.01),kernel_regularizer=reg2,padding='same'))
model.add(BatchNormalization(axis=-1)) 
model.add(MaxPooling2D(pool_size=(2, 2)))  # reduces to 2x2x3x(8*num_filters)
model.add(Dropout(drop_conv2))

model.add(Flatten())
model.add(Dense(1024, activation=k.layers.LeakyReLU(alpha=0.01),kernel_regularizer=reg2))
model.add(Dropout(drop_dense2))
model.add(Dense(512, activation=k.layers.LeakyReLU(alpha=0.01),kernel_regularizer=reg2))
model.add(Dropout(drop_dense2))
model.add(BatchNormalization(axis=-1))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer=opt)
model.summary()

In [ ]:
# train without image augmentation
history2=model.fit(x_train,y_train,batch_size=128,epochs=5,
                   validation_data=(x_test, y_test),shuffle=True
)


In [ ]:
# train with image augmentation
history2=model.fit(datagen.flow(x_train,y_train),batch_size=128,epochs=2,
                   validation_data=(x_test, y_test),shuffle=True
)


In [ ]:
plothist(history2)

In [ ]:
model.evaluate(x_test,y_test,batch_size=32)